In [11]:
import os
import settings
import pandas as pd
import operator

from sklearn.tree import DecisionTreeRegressor

def read_data():
    df = pd.read_excel(os.path.join(settings.PROCESSED_DIR, "all_with_liwc_segmented.xls"), encoding="ISO-8859-1")
    return df

def sort_important_features(df):
    dt = DecisionTreeRegressor()
    predictors = df.columns.tolist()
    predictors = [p for p in predictors if p not in settings.NON_PREDICTORS]
    dt.fit(df[predictors], df[settings.TARGET])
    predictions = dt.predict(df[predictors])
    results = {name: score for name, score in zip(predictors, dt.feature_importances_)}
    sorted_results = sorted(results.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_results
    accuracy = dt.score(df[predictors], df[settings.TARGET])
    print("Accuracy: {}".format(accuracy))

In [12]:
df = read_data()

In [13]:
sort_important_features(df)

[('posemo_1h', 0.24356271948886576),
 ('body', 0.048354052596182653),
 ('number', 0.045886074358241519),
 ('IngroupVice', 0.042830124272925818),
 ('focusfuture', 0.039407695482444938),
 ('focuspast', 0.037671096927211371),
 ('IngroupVirtue', 0.026620154176547617),
 ('quant', 0.023379077174485063),
 ('relig', 0.021961023365125693),
 ('family', 0.020906827726456793),
 ('ppron', 0.020835632532224268),
 ('focuspresent', 0.020652378189289198),
 ('Purity', 0.016833531558779504),
 ('MoralityGeneral', 0.016711140384629543),
 ('negemo_change_h', 0.015681562010198623),
 ('posemo', 0.014808093467130521),
 ('posemo_4q', 0.014366983224136222),
 ('HarmVirtue', 0.01364509097786454),
 ('negate', 0.011786744408827435),
 ('bio', 0.01136153087201313),
 ('differ', 0.011360139495282527),
 ('reward', 0.010888041625265505),
 ('social', 0.010600181256661109),
 ('money', 0.010550365029988731),
 ('discrep', 0.010444267249150664),
 ('risk', 0.010244329430560048),
 ('Ingroup', 0.0082530221949987988),
 ('auxverb',

In [10]:
sort_important_features(df)

[('body', 0.29089387718470672),
 ('number', 0.04427302999511825),
 ('you', 0.038821910851747674),
 ('focuspast', 0.033896974362081937),
 ('Ingroup', 0.033630899156230479),
 ('focuspresent', 0.028574963042394954),
 ('focusfuture', 0.025369077714324836),
 ('quant', 0.023892495468566831),
 ('relig', 0.021715909285407908),
 ('motion', 0.019654990636341265),
 ('ppron', 0.018896402966552664),
 ('shehe', 0.018578565464326083),
 ('MoralityGeneral', 0.018248205503494565),
 ('negemo_change_h', 0.016039787934427195),
 ('posemo', 0.015173686172934202),
 ('HarmVirtue', 0.013154242265729308),
 ('prep', 0.011951680527830681),
 ('negate', 0.011780994047845619),
 ('posemo_4q', 0.011629773080020535),
 ('differ', 0.011558822993386167),
 ('bio', 0.011421956329098114),
 ('money', 0.010230493492770925),
 ('discrep', 0.010033562872714601),
 ('assent', 0.0094218081813901011),
 ('pronoun', 0.0091734141680456446),
 ('posemo_change_h', 0.0081488956710381154),
 ('auxverb', 0.0079542406585650996),
 ('affect', 0.00

In [14]:
sort_important_features(df)

[('ppron', 0.26261937607589003),
 ('body', 0.047621944745011406),
 ('number', 0.044879298548985681),
 ('quant', 0.04175439617876596),
 ('ipron', 0.038941005417733487),
 ('focuspast', 0.03754273431815007),
 ('IngroupVirtue', 0.026701549888033051),
 ('focusfuture', 0.025698619915904157),
 ('focuspresent', 0.025419342783211181),
 ('relig', 0.023772159353482106),
 ('relativ', 0.019433097510991423),
 ('MoralityGeneral', 0.016578690228721268),
 ('posemo', 0.014741797782837031),
 ('negemo_change_h', 0.014152259817449301),
 ('HarmVirtue', 0.013508593879724569),
 ('differ', 0.012614819192825762),
 ('prep', 0.012451411290019178),
 ('negate', 0.012068242700433237),
 ('posemo_4q', 0.01185915514336696),
 ('bio', 0.011323505517709503),
 ('money', 0.0098678671271770282),
 ('assent', 0.0094251322197605372),
 ('certain', 0.0092550806982936318),
 ('space', 0.0088037509649248231),
 ('auxverb', 0.0081104431168363932),
 ('anx_1h', 0.0079217096437584622),
 ('Ingroup', 0.0076099424326522827),
 ('cogproc', 0.

## Random Forest 

In [15]:
import os
import settings
import pandas as pd
import operator
from sklearn.ensemble import RandomForestRegressor

In [16]:
def read_data():
    df = pd.read_excel(os.path.join(settings.PROCESSED_DIR, "all_with_liwc_segmented.xls"), encoding="ISO-8859-1")
    return df

def sort_important_features(df):
    rf = RandomForestRegressor()
    predictors = df.columns.tolist()
    predictors = [p for p in predictors if p not in settings.NON_PREDICTORS]
    rf.fit(df[predictors], df[settings.TARGET])
    predictions = rf.predict(df[predictors])
    results = {name: score for name, score in zip(predictors, rf.feature_importances_)}
    sorted_results = sorted(results.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_results
    accuracy = rf.score(df[predictors], df[settings.TARGET])
    print("Accuracy: {}".format(accuracy))

In [ ]:
df = read_data()

In [20]:

sort_important_features(df)

[('negate', 0.07685770363432172),
 ('they', 0.060144645138372477),
 ('leisure', 0.059842315599297514),
 ('body', 0.047107756251783782),
 ('focusfuture', 0.037167025921783357),
 ('certain', 0.034090261848013073),
 ('relig', 0.033974782866198081),
 ('bio', 0.020482222503385612),
 ('auxverb', 0.02040273484537055),
 ('social', 0.01983851981024121),
 ('MoralityGeneral', 0.016984711012769559),
 ('posemo', 0.016396519720402496),
 ('focuspast', 0.016234862798289305),
 ('anger_3q', 0.015705707594225925),
 ('ipron', 0.015162339178368189),
 ('drives', 0.014056049958311043),
 ('Moral', 0.011647238192472075),
 ('money', 0.011629979082169381),
 ('tentat', 0.010853640166697457),
 ('differ', 0.010713601538973381),
 ('number', 0.010583168470271622),
 ('time', 0.010516517266439099),
 ('anx_4q', 0.010365255136557451),
 ('affect', 0.01020492108786818),
 ('insight', 0.01008134059426756),
 ('affect_change_h', 0.0096785321601828671),
 ('anx_3q', 0.0094961764416660252),
 ('ingest', 0.0094197833474371391),
 ('

In [21]:
sort_important_features(df)

[('leisure', 0.083145240658971289),
 ('negate', 0.076839337459097656),
 ('power', 0.067007288310190358),
 ('focuspast', 0.046444638816215217),
 ('adverb', 0.040563231357900902),
 ('relig', 0.036680886678617672),
 ('work', 0.026755769052789306),
 ('focusfuture', 0.025959422269963651),
 ('negemo_2q', 0.022638583954957985),
 ('MoralityGeneral', 0.020731588888174887),
 ('cogproc', 0.01984245830449282),
 ('auxverb', 0.018854490695330701),
 ('number', 0.016480601198442545),
 ('motion', 0.016350726005448627),
 ('focuspresent', 0.013719256179956743),
 ('anx_4q', 0.011592457726777682),
 ('tentat', 0.010460207094794387),
 ('sad_1h', 0.0098170962261137449),
 ('conj', 0.0097930023379002532),
 ('certain', 0.0092894566352310082),
 ('IngroupVice', 0.0091528783963771392),
 ('adj', 0.0090970702896746442),
 ('bio', 0.0090802212178031239),
 ('interrog', 0.0084853009548128463),
 ('FairnessVice', 0.0083055605455010194),
 ('posemo', 0.0079651710567887092),
 ('achieve', 0.007957660324418879),
 ('social', 0.0

In [22]:
sort_important_features(df)

[('negate', 0.054330037715501475),
 ('number', 0.054237849359518787),
 ('relig', 0.036134560733292277),
 ('focusfuture', 0.033343738175779701),
 ('tentat', 0.033018495698908294),
 ('leisure', 0.032483989933626403),
 ('conj', 0.028567016673758917),
 ('body', 0.025707248043350017),
 ('Analytic', 0.023876892286335639),
 ('interrog', 0.022717383800067346),
 ('auxverb', 0.021123229150911749),
 ('MoralityGeneral', 0.019493293133491168),
 ('Fairness', 0.018778811975883981),
 ('anger_2q', 0.017585938546624581),
 ('focuspast', 0.017231296939465263),
 ('power', 0.015272302546985504),
 ('focuspresent', 0.014623598508479732),
 ('reward', 0.013318212721992054),
 ('relativ', 0.013256403440814574),
 ('posemo', 0.012808071470646437),
 ('motion', 0.012080669491735828),
 ('posemo_2h', 0.01156425291579884),
 ('percept', 0.010989282875064284),
 ('adverb', 0.010726748824180442),
 ('informal', 0.010676835678806833),
 ('social', 0.010578299602917715),
 ('anx_4q', 0.010440216112236927),
 ('bio', 0.01031258816

## Linear Regression

In [23]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

/Users/owentemple/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [79]:
def create_summary(df):
    lr = LinearRegression()
    predictors = df.columns.tolist()
    predictors = ['relig', 'MoralityGeneral', 'cogproc', 'focusfuture', 'cogproc']
    lr.fit(df[predictors], df['norm_unconvincing'])

    X = df[predictors]
    X2 = sm.add_constant(X)
    est = sm.OLS(df['norm_unconvincing'], X2)
    est2 = est.fit()
    print(est2.summary())

In [80]:
create_summary(df)

                            OLS Regression Results                            
Dep. Variable:      norm_unconvincing   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     24.03
Date:                Thu, 11 Jan 2018   Prob (F-statistic):           1.63e-19
Time:                        17:47:25   Log-Likelihood:                -13938.
No. Observations:                2406   AIC:                         2.789e+04
Df Residuals:                    2401   BIC:                         2.792e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -12.2575      9.184     